# Read Data

In [ ]:
import pandas as pd
df = pd.read_csv('Alcohol_Sales.csv', index_col = 'DATE', parse_dates = True)
df.columns = ['Sales']
df.head()

FileNotFoundError: ignored

# Visualize

In [ ]:
df['Sales'].plot(figsize = (12,6))

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
df.index.freq = 'DS'

result = seasonal_decompose(df['Sales'])

In [ ]:
result.trend.plot(figsize = (12,4))

In [ ]:
result.seasonal.plot(figsize = (12,4))

In [ ]:
result.resid.plot(figsize = (12,4))

# train/test split

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
print(df.index.min())
print(df.index.max())

In [ ]:
test_size = 36
split_idx = len(df) - test_size

train = df.iloc[:split_idx]
test = df.iloc[split_idx:]
print(train.shape)
print(test.shape)

# scaling

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(train)
scaled_train = scaler.transform(train)
scaled_test = scaler.transform(test)

### Test TimeseriesGenerator(학습용)

In [ ]:
import tensorflow as tf
window_size = 2
generator = tf.keras.preprocessing.sequence.TimeseriesGenerator(scaled_train, scaled_train, length = window_size, batch_size = 1)

In [ ]:
generator

In [ ]:
scaled_train[:10].flatten()

In [ ]:
for i in range(10):
  X, y = generator[i]
  print(f'input: {X.flatten()}, target: {y}')

### TimeseriesGenerator

In [ ]:
# 12개월 데이터를 input -> 1개월 데이터를 예측(target)
window_size = 12
generator = tf.keras.preprocessing.sequence.TimeseriesGenerator(scaled_train, scaled_train, length = window_size, batch_size = 1)

# modeling(compile, fit)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

model = Sequential()
model.add(LSTM(100, activation = 'relu', input_shape = (window_size, 1)))
model.add(Dense(1))
model.compile(optimizer = 'adam', loss = 'mse')
model.summary()

In [ ]:
model.fit(generator, epochs = 50)

# model evaluate(평가)

In [ ]:
print(train.index.min())
print(train.index.max())
print(test.index.min())
print(test.index.max())

In [ ]:
# 12개월의 데이터가 필요하다.
# test data / 필요한 데이터
# 2016-02-01 -> 2015-02-01 ~ 2016-01-01
# 2019-01-01 -> 2018-01-01 ~ 2018-12-01
# 한번에 모든 test데이터를 검증하는게 가능한가?

In [ ]:
# 2016-02-01
batch = scaled_train[-12:].reshape(1, window_size, 1)
batch.shape

In [ ]:
model.predict(batch)

In [ ]:
scaled_test[0]

In [ ]:
# 2016-03-01
scaled_train[-11:]

In [ ]:
import numpy as np
batch2 = np.append(scaled_train[-11:], [scaled_test[0]])
batch2.shape
print(batch2)

In [ ]:
# 모델의 결과를 for문으로 작성하고 scaled_test 비교